# Technical test for the data science internship @Hyperlex

In [46]:
import os
import json
import numpy as np
import pandas as pd
import nltk

In [48]:
!find . -iname *checkpoint* -type d -exec rm -r {} +  

FIND: Parameter format not correct


In [49]:
rootdir = 'dataset'
json_files = [os.path.join(dirpath, file) for dirpath,_,files in os.walk(rootdir) for file in files if file.endswith('json')]

data = []
for file in json_files:
    with open(file) as f:
        data.append(json.load(f)['clauses'][1:])  # starting from one to skip the summary since they contain different informations 
                                                  # and could affect the classification
articles = [article for contract in data for article in contract ]

In [51]:
test = pd.DataFrame(articles)